In [ ]:
# GROUP BY + CASE WHEN

In [6]:
import duckdb

# 1. DuckDB 연결
con = duckdb.connect(database=':memory:', read_only=False)

# 2. CREATE TABLE 및 INSERT INTO
print("--- 1. 데이터 준비 (CREATE/INSERT) ---")
con.sql("""
CREATE TABLE Sales (
    판매ID INTEGER PRIMARY KEY,
    지역 VARCHAR,
    상품 VARCHAR,
    수량 INTEGER,
    단가 INTEGER,
    매출액 INTEGER
);
""")
con.sql("""
INSERT INTO Sales (판매ID, 지역, 상품, 수량, 단가, 매출액) VALUES
(1, '서울', '노트북', 5, 1200000, 6000000),
(2, '부산', '태블릿', 10, 500000, 5000000),
(3, '서울', '모니터', 3, 300000, 900000),
(4, '제주', '노트북', 2, 1200000, 2400000),
(5, '부산', '노트북', 7, 1200000, 8400000),
(6, '서울', '태블릿', 8, 500000, 4000000),
(7, '제주', '모니터', 1, 300000, 300000),
(8, '서울', '노트북', 4, 1200000, 4800000);
""")

# 3. SELECT로 데이터 확인
print("\n--- 2. 전체 데이터 확인 ---")
con.sql("SELECT * FROM Sales;").show()

# 4. GROUP BY + CASE WHEN (조건부 집계/피벗팅)
print("\n--- 3. GROUP BY + CASE WHEN (지역별 상품 매출 피벗) ---")
con.sql("""
SELECT
    상품,
    SUM(CASE WHEN 지역 = '서울' THEN 매출액 ELSE 0 END) AS 서울_총매출,
    SUM(CASE WHEN 지역 = '부산' THEN 매출액 ELSE 0 END) AS 부산_총매출,
    SUM(CASE WHEN 지역 = '제주' THEN 매출액 ELSE 0 END) AS 제주_총매출
FROM
    Sales
GROUP BY
    상품
ORDER BY
    상품;
""").show()

# 5. GROUP BY + HAVING (집계 후 필터링)
print("\n--- 4. GROUP BY + HAVING (평균 수량 5개 이상인 지역 필터링) ---")
con.sql("""
SELECT
    지역,
    AVG(수량) AS 평균_판매_수량
FROM
    Sales
GROUP BY
    지역
HAVING
    평균_판매_수량 >= 5
ORDER BY
    평균_판매_수량 DESC;
""").show()

con.close()

--- 1. 데이터 준비 (CREATE/INSERT) ---

--- 2. 전체 데이터 확인 ---
┌────────┬─────────┬─────────┬───────┬─────────┬─────────┐
│ 판매ID │  지역   │  상품   │ 수량  │  단가   │ 매출액  │
│ int32  │ varchar │ varchar │ int32 │  int32  │  int32  │
├────────┼─────────┼─────────┼───────┼─────────┼─────────┤
│      1 │ 서울    │ 노트북  │     5 │ 1200000 │ 6000000 │
│      2 │ 부산    │ 태블릿  │    10 │  500000 │ 5000000 │
│      3 │ 서울    │ 모니터  │     3 │  300000 │  900000 │
│      4 │ 제주    │ 노트북  │     2 │ 1200000 │ 2400000 │
│      5 │ 부산    │ 노트북  │     7 │ 1200000 │ 8400000 │
│      6 │ 서울    │ 태블릿  │     8 │  500000 │ 4000000 │
│      7 │ 제주    │ 모니터  │     1 │  300000 │  300000 │
│      8 │ 서울    │ 노트북  │     4 │ 1200000 │ 4800000 │
└────────┴─────────┴─────────┴───────┴─────────┴─────────┘


--- 3. GROUP BY + CASE WHEN (지역별 상품 매출 피벗) ---
┌─────────┬─────────────┬─────────────┬─────────────┐
│  상품   │ 서울_총매출 │ 부산_총매출 │ 제주_총매출 │
│ varchar │   int128    │   int128    │   int128    │
├─────────┼─────────────┼───────────

In [ ]:
import duckdb

# 1. DuckDB 연결
con = duckdb.connect(database=':memory:', read_only=False)

# 2. CREATE TABLE 및 INSERT INTO (데이터 준비)
print("--- 1. Project 데이터 준비 (CREATE/INSERT) ---")
con.sql("""
CREATE TABLE Project (
    기록ID INTEGER PRIMARY KEY,
    팀명 VARCHAR,
    프로젝트명 VARCHAR,
    투입시간 INTEGER,
    시간당비용 INTEGER,
    총비용 INTEGER
);
""")
con.sql("""
INSERT INTO Project (기록ID, 팀명, 프로젝트명, 투입시간, 시간당비용, 총비용) VALUES
(101, '알파팀', '웹사이트 개발', 40, 50000, 2000000),
(102, '베타팀', '모바일 앱 개발', 60, 60000, 3600000),
(103, '알파팀', '데이터 분석', 20, 55000, 1100000),
(104, '감마팀', '웹사이트 개발', 50, 40000, 2000000),
(105, '베타팀', '모바일 앱 개발', 30, 60000, 1800000),
(106, '알파팀', '웹사이트 개발', 10, 50000, 500000),
(107, '감마팀', '데이터 분석', 70, 45000, 3150000),
(108, '베타팀', '데이터 분석', 40, 65000, 2600000);
""")

# 3. SELECT로 데이터 확인
print("\n--- 2. 전체 Project 데이터 확인 ---")
con.sql("SELECT * FROM Project;").show()

# 4. GROUP BY + CASE WHEN (조건부 집계/피벗팅)
# 팀별로 '프로젝트 유형별' 투입 시간 합계를 비교
print("\n--- 3. GROUP BY + CASE WHEN (팀별 프로젝트 유형별 투입 시간 비교) ---")
con.sql("""
SELECT
    팀명,
    -- 웹사이트 개발에 투입된 총 시간
    SUM(CASE WHEN 프로젝트명 = '웹사이트 개발' THEN 투입시간 ELSE 0 END) AS 웹사이트_시간,
    -- 모바일 앱 개발에 투입된 총 시간
    SUM(CASE WHEN 프로젝트명 = '모바일 앱 개발' THEN 투입시간 ELSE 0 END) AS 모바일앱_시간,
    -- 데이터 분석에 투입된 총 시간
    SUM(CASE WHEN 프로젝트명 = '데이터 분석' THEN 투입시간 ELSE 0 END) AS 데이터분석_시간,
    SUM(투입시간) AS 총_투입_시간
FROM
    Project
GROUP BY
    팀명
ORDER BY
    총_투입_시간 DESC;
""").show()

# 5. GROUP BY + HAVING (집계 후 필터링)
# 팀별 '평균 총비용'을 계산한 후, 평균 비용이 2,500,000원 이상인 팀만 선별
print("\n--- 4. GROUP BY + HAVING (평균 총비용 2,500,000 이상인 팀 필터링) ---")
con.sql("""
SELECT
    팀명,
    COUNT(기록ID) AS 프로젝트_기록_건수,
    AVG(총비용) AS 평균_총비용
FROM
    Project
GROUP BY
    팀명
-- **핵심:** 집계된 '평균_총비용'이 2,500,000원을 초과하는 그룹만 선택
HAVING
    평균_총비용 >= 2500000
ORDER BY
    평균_총비용 DESC;
""").show()


con.close()

--- 1. Project 데이터 준비 (CREATE/INSERT) ---

--- 2. 전체 Project 데이터 확인 ---
┌────────┬─────────┬────────────────┬──────────┬────────────┬─────────┐
│ 기록ID │  팀명   │   프로젝트명   │ 투입시간 │ 시간당비용 │ 총비용  │
│ int32  │ varchar │    varchar     │  int32   │   int32    │  int32  │
├────────┼─────────┼────────────────┼──────────┼────────────┼─────────┤
│    101 │ 알파팀  │ 웹사이트 개발  │       40 │      50000 │ 2000000 │
│    102 │ 베타팀  │ 모바일 앱 개발 │       60 │      60000 │ 3600000 │
│    103 │ 알파팀  │ 데이터 분석    │       20 │      55000 │ 1100000 │
│    104 │ 감마팀  │ 웹사이트 개발  │       50 │      40000 │ 2000000 │
│    105 │ 베타팀  │ 모바일 앱 개발 │       30 │      60000 │ 1800000 │
│    106 │ 알파팀  │ 웹사이트 개발  │       10 │      50000 │  500000 │
│    107 │ 감마팀  │ 데이터 분석    │       70 │      45000 │ 3150000 │
│    108 │ 베타팀  │ 데이터 분석    │       40 │      65000 │ 2600000 │
└────────┴─────────┴────────────────┴──────────┴────────────┴─────────┘


--- 3. GROUP BY + CASE WHEN (팀별 프로젝트 유형별 투입 시간 비교) ---
┌─────────┬──────────────

In [10]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

print("--- 1. CustomerActivity 테이블 생성 및 삽입 ---")
con.sql("""
CREATE TABLE CustomerActivity (
    거래ID INTEGER PRIMARY KEY,
    고객ID INTEGER,
    세그먼트 VARCHAR, -- 'Standard', 'Premium'
    거래금액 INTEGER,
    반복구매여부 BOOLEAN -- T: 반복구매, F: 최초구매/단건구매
);
""")
con.sql("""
INSERT INTO CustomerActivity (거래ID, 고객ID, 세그먼트, 거래금액, 반복구매여부) VALUES
(301, 10, 'Premium', 800000, TRUE),
(302, 10, 'Premium', 200000, TRUE),
(303, 20, 'Standard', 300000, FALSE),
(304, 30, 'Premium', 1200000, FALSE),
(305, 30, 'Premium', 100000, FALSE),
(306, 40, 'Standard', 50000, TRUE),
(307, 40, 'Standard', 60000, TRUE),
(308, 50, 'Premium', 400000, FALSE),
(309, 50, 'Premium', 700000, FALSE);
""")
con.sql("SELECT * FROM CustomerActivity;").show()

--- 1. CustomerActivity 테이블 생성 및 삽입 ---
┌────────┬────────┬──────────┬──────────┬──────────────┐
│ 거래ID │ 고객ID │ 세그먼트 │ 거래금액 │ 반복구매여부 │
│ int32  │ int32  │ varchar  │  int32   │   boolean    │
├────────┼────────┼──────────┼──────────┼──────────────┤
│    301 │     10 │ Premium  │   800000 │ true         │
│    302 │     10 │ Premium  │   200000 │ true         │
│    303 │     20 │ Standard │   300000 │ false        │
│    304 │     30 │ Premium  │  1200000 │ false        │
│    305 │     30 │ Premium  │   100000 │ false        │
│    306 │     40 │ Standard │    50000 │ true         │
│    307 │     40 │ Standard │    60000 │ true         │
│    308 │     50 │ Premium  │   400000 │ false        │
│    309 │     50 │ Premium  │   700000 │ false        │
└────────┴────────┴──────────┴──────────┴──────────────┘



In [11]:
print("\n--- 3. 고액 고객 중 충성도 낮은 상위 3명 선별 ---")
con.sql("""
SELECT
    고객ID,
    세그먼트,
    SUM(거래금액) AS 총_거래액,
    COUNT(거래ID) AS 총_거래_건수,

    -- 1. 조건부 집계 (CASE WHEN): 50만원 초과 고가 거래 건수 계산
    SUM(CASE WHEN 거래금액 > 500000 THEN 1 ELSE 0 END) AS 고가_거래_건수,

    -- 2. 반복 구매율 계산 (BOOLEAN::INTEGER 형 변환으로 오류 해결)
    AVG(반복구매여부::INTEGER) * 100 AS 반복구매율_퍼센트
FROM
    CustomerActivity
GROUP BY
    고객ID, 세그먼트
-- 3. 집계 후 필터링 (HAVING): 총 거래액 100만원 이상 AND 반복구매율 50% 미만
HAVING
    총_거래액 >= 1000000 AND 반복구매율_퍼센트 < 50
ORDER BY
    총_거래액 DESC -- 4. 총 거래액 높은 순 정렬 (ORDER BY)
LIMIT
    3; -- 4. 상위 3명만 출력 (LIMIT)
""").show()

con.close()


--- 3. 고액 고객 중 충성도 낮은 상위 3명 선별 ---
┌────────┬──────────┬───────────┬──────────────┬────────────────┬───────────────────┐
│ 고객ID │ 세그먼트 │ 총_거래액 │ 총_거래_건수 │ 고가_거래_건수 │ 반복구매율_퍼센트 │
│ int32  │ varchar  │  int128   │    int64     │     int128     │      double       │
├────────┼──────────┼───────────┼──────────────┼────────────────┼───────────────────┤
│     30 │ Premium  │   1300000 │            2 │              1 │               0.0 │
│     50 │ Premium  │   1100000 │            2 │              1 │               0.0 │
└────────┴──────────┴───────────┴──────────────┴────────────────┴───────────────────┘



In [ ]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

# 0. 기존 테이블이 있으면 삭제
con.sql("DROP TABLE IF EXISTS EmployeePerformance;")

# 1. CREATE TABLE 및 INSERT INTO (데이터 준비)
print("--- 1. EmployeePerformance 테이블 생성 및 삽입 ---")
con.sql("""
CREATE TABLE EmployeePerformance (
    직원ID INTEGER PRIMARY KEY,
    부서명 VARCHAR,
    총비용 INTEGER,
    성과점수 INTEGER
);
""")
con.sql("""
INSERT INTO EmployeePerformance (직원ID, 부서명, 총비용, 성과점수) VALUES
(401, '마케팅', 6000000, 85),
(402, '마케팅', 5500000, 78),
(403, '개발', 9000000, 95),
(404, '개발', 8500000, 90),
(405, '영업', 5000000, 60),
(406, '영업', 4500000, 55),
(407, 'HR', 7000000, 80),
(408, 'HR', 6500000, 65),
(409, '영업', 8000000, 70),
(410, '개발', 9500000, 60);
""")

# 2. SELECT로 데이터 확인
con.sql("SELECT * FROM EmployeePerformance;").show()

# 3. 분석 쿼리: 비효율 부서 (고비용/저성과) 상위 2개 선별
print("\n--- 3. 비효율 부서 (고비용/저성과) 상위 2개 선별 ---")
con.sql("""
SELECT
    부서명,
    COUNT(직원ID) AS 총_직원_수,
    AVG(총비용) AS 평균_직원_총비용,
    AVG(성과점수) AS 평균_부서_성과,
    
    SUM(CASE WHEN 성과점수 > 80 THEN 1 ELSE 0 END) AS 고성과자_수
FROM
    EmployeePerformance
GROUP BY
    부서명
HAVING
    평균_직원_총비용 >= 6000000 AND 평균_부서_성과 < 75
ORDER BY
    평균_직원_총비용 DESC
LIMIT
    2;
""").show()

con.close()

--- 1. EmployeePerformance 테이블 생성 및 삽입 ---
┌────────┬─────────┬─────────┬──────────┐
│ 직원ID │ 부서명  │ 총비용  │ 성과점수 │
│ int32  │ varchar │  int32  │  int32   │
├────────┼─────────┼─────────┼──────────┤
│    401 │ 마케팅  │ 6000000 │       85 │
│    402 │ 마케팅  │ 5500000 │       78 │
│    403 │ 개발    │ 9000000 │       95 │
│    404 │ 개발    │ 8500000 │       90 │
│    405 │ 영업    │ 5000000 │       60 │
│    406 │ 영업    │ 4500000 │       55 │
│    407 │ HR      │ 7000000 │       80 │
│    408 │ HR      │ 6500000 │       65 │
│    409 │ 영업    │ 8000000 │       70 │
│    410 │ 개발    │ 9500000 │       60 │
├────────┴─────────┴─────────┴──────────┤
│ 10 rows                     4 columns │
└───────────────────────────────────────┘


--- 3. 비효율 부서 (고비용/저성과) 상위 2개 선별 ---
┌─────────┬────────────┬──────────────────┬────────────────┬─────────────┐
│ 부서명  │ 총_직원_수 │ 평균_직원_총비용 │ 평균_부서_성과 │ 고성과자_수 │
│ varchar │   int64    │      double      │     double     │   int128    │
├─────────┼────────────┼──────────